In [127]:
import pandas as pd
import config
import os
import csv
import sys

In [29]:
csv.field_size_limit(sys.maxsize)

131072

In [130]:
os.listdir(os.path.join('../',config.data_dir, 'chunks/'))

['patents_2019_chunk_000013.csv',
 'patents_2019_chunk_000000.csv',
 'patents_2019_chunk_000001.csv',
 'patents_2019_chunk_000002.csv',
 'patents_2019_chunk_000003.csv',
 'patents_2019_chunk_000004.csv',
 'patents_2019_chunk_000005.csv',
 'patents_2019_chunk_000006.csv',
 'patents_2019_chunk_000007.csv',
 'patents_2019_chunk_000008.csv',
 'patents_2019_chunk_000009.csv',
 'patents_2019_chunk_000010.csv',
 'patents_2019_chunk_000011.csv',
 'patents_2019_chunk_000012.csv',
 'patents_2019_chunk_000014.csv',
 'patents_2019_chunk_000015.csv',
 'patents_2019_chunk_000016.csv',
 'patents_2019_chunk_000017.csv',
 'patents_2019_chunk_000018.csv',
 'patents_2019_chunk_000019.csv',
 'patents_2019_chunk_000020.csv',
 'patents_2019_chunk_000021.csv',
 'patents_2019_chunk_000022.csv',
 'patents_2019_chunk_000023.csv',
 'patents_2019_chunk_000024.csv',
 'patents_2019_chunk_000025.csv',
 'patents_2019_chunk_000026.csv',
 'patents_2019_chunk_000027.csv',
 'patents_2019_chunk_000028.csv',
 'patents_2019

In [128]:
class_names = config.labels_list
features = Features({'text': Value('string'), 'label': ClassLabel(names=class_names)})
data_files = {"train":os.path.join(config.data_dir, 'example/train.csv'), "test":os.path.join(config.data_dir, 'example/test.csv')}
dataset = load_dataset('csv', data_files=data_files, features=features)

for chunk in patents:
    # Combine patent with respective section info.
    data = chunk.merge(ipcr, how='left', on='patent_id')

    # Replace the letters with integers to create a suitable training input.
    data.replace({'section':config.label2id}, inplace=True)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7}

In [113]:
test = pd.read_csv(os.path.join('../',config.data_dir, 'chunks/patents_2019_chunk_000000.csv'),
    skiprows=50000,
    nrows=10,
    header=None,
    names = ['id', 'patent_id','text','length'],

    )
test


,id,patent_id,text,length
0,DETAILED DESCRIPTION OF THE INVENTION\n\nIn th...,6,NaN,NaN


In [109]:
patents = pd.read_csv(os.path.join('../',config.data_dir, 'detail_desc_text_'+config.patents_year+'.tsv'),
    sep="\t",
    encoding='utf8',
    skiprows=319054,
    engine='c',
    nrows=10,
    )
patents

EmptyDataError: No columns to parse from file

In [105]:
patents = pd.read_csv(os.path.join('../',config.data_dir, 'detail_desc_text_'+config.patents_year+'.tsv'),
    sep="\t",
    encoding='utf8',
    on_bad_lines='warn',
    encoding_errors='ignore',
   # skiprows=[1,1,1,0], # skiprows=319052,
    engine='c',
    nrows=10,
    header=None,
    names = ['id', 'patent_id','text','length'],
    usecols=['patent_id', 'text'],
    dtype={'patent_id':object, 'text':object},
    )
patents

,patent_id,text
0,patent_id,text
1,10165721,DESCRIPTION OF THE PREFERRED EMBODIMENTS\n\nWi...
2,10165722,DETAILED DESCRIPTION\n\nHuman beings have cert...
3,10165723,"DETAILED DESCRIPTION\n\nReferring toFIGS. 1, 1..."
4,10165724,DETAILED DESCRIPTION OF THE PREFERRED EMBODIME...
5,10165725,DETAILED DESCRIPTION\n\nBefore describing the ...
6,10165726,Corresponding reference characters indicate co...
7,10165727,DETAILED DESCRIPTION OF PREFERRED EMBODIMENTS ...
8,10165728,DETAILED DESCRIPTION OF THE PREFERRED EMBODIME...
9,10165729,DETAILED DESCRIPTION\n\nReference will now be ...


In [ ]:
# hatalı kayııtan bir önceki id 10487259
# hatalı kayıt satur sayısı 319052

In [48]:
patentspatents = pd.read_csv(os.path.join('../',config.data_dir, 'detail_desc_text_'+config.patents_year+'.tsv'),
    sep="\t",
    encoding='utf8',
    on_bad_lines='warn',
    encoding_errors='ignore',
    skiprows=319054,
    engine='c',
    nrows=3,
    header=None,
    names = ['id', 'patent_id','text','length'],
    usecols=['patent_id', 'text'],
    dtype={'patent_id':object, 'text':object},
    )
patents

,id,patent_id,text,length
0,3a88fd0f-fb74-11ea-a344-121df0c29c1e,10487259,DETAILED DESCRIPTION\n\nReference will now be ...,78570


In [61]:
patents

,id,patent_id,text,length
0,Synthesis,NaN,NaN,NaN
1,A further aspect of the invention relates to a...,NaN,NaN,NaN
2,The reaction is preferably performed in the pr...,NaN,NaN,NaN
3,In one embodiment the process for preparing a ...,NaN,NaN,NaN
4,The reaction is preferably performed under mit...,NaN,NaN,NaN
...,...,...,...,...
49995,"In the arrangement300according toFIG. 3, an em...",NaN,NaN,NaN
49996,In one embodiment of the method according to t...,NaN,NaN,NaN
49997,A deviation of the anticipated target value of...,NaN,NaN,NaN
49998,"The recording of the sensor values, storing th...",NaN,NaN,NaN


In [26]:
df.loc[299995:]

,text,label
299995,DETAILED DESCRIPTION\n\nThe present invention ...,6
299996,DETAILED DESCRIPTION OF THE DISCLOSURE\n\nEmbo...,6
299997,DETAILED DESCRIPTION OF PREFERRED EMBODIMENTS\...,7
299998,DETAILED DESCRIPTION\n\nReference will now be ...,6
299999,DETAILED DESCRIPTION\n\nThe following detailed...,6
300000,DETAILED DESCRIPTION\n\nThe present invention ...,6
300001,DETAILED DESCRIPTION\n\nDisclosed herein is a ...,6
300002,DETAILED DESCRIPTION\n\nThe following detailed...,6
300003,DETAILED DESCRIPTION\n\nThe following detailed...,6
300004,DETAILED DESCRIPTION OF THE INVENTION\n\nThe s...,6


In [24]:
cf = pd.read_csv('../data/patentsview/chunks/patents_2019_chunk_000005.csv')

In [33]:
df[df["text"].duplicated(keep="last")]

,text,label
15,DETAILED DESCRIPTION\n\nIn accordance with the...,0
107,DETAILED DESCRIPTION\n\nFIG. 1illustrates an i...,0
258,DETAILED DESCRIPTION\n\nIn accordance with exe...,0
285,"Referring first toFIG. 1, an ultrasound system...",0
470,DETAILED DESCRIPTION\n\nThe present invention ...,0
...,...,...
289291,Reference will now be made in detail to the pr...,1
291793,DETAILED DESCRIPTION\n\nDescribed herein are m...,6
294162,DETAILED DESCRIPTION\n\nFIG. 1is a schematic d...,7
294594,DETAILED DESCRIPTION\n\nThis disclosure is bas...,7


In [41]:
xf = pd.read_csv('../data/patentsview/detail_desc_text_2019.tsv', sep='\t', nrows=20000, usecols=['patent_id', 'text'])

In [42]:
xf[xf['text'] == df.loc[15]['text']]

,patent_id,text
15,10165736,DETAILED DESCRIPTION\n\nIn accordance with the...
6532,10172299,DETAILED DESCRIPTION\n\nIn accordance with the...


In [44]:
ipcr = pd.read_csv('../data/patentsview/ipcr.tsv', sep="\t", usecols=['patent_id','section'])

In [45]:
ipcr[ipcr['patent_id']== '10165736']

,patent_id,section
1276152,10165736,A
5699882,10165736,A
6489837,10165736,A
6987628,10165736,G
7935397,10165736,G
15065440,10165736,G
16468120,10165736,A


In [46]:
ipcr[ipcr['patent_id']== '10172299']

,patent_id,section
2436044,10172299,G
6948426,10172299,A
10329172,10172299,A
10983217,10172299,G
15026684,10172299,G
15503788,10172299,A
16051504,10172299,A


In [36]:
df[df['text'] == df.loc[15]['text']]

,text,label
15,DETAILED DESCRIPTION\n\nIn accordance with the...,0
6532,DETAILED DESCRIPTION\n\nIn accordance with the...,6


In [40]:
df.loc[15]['text']

"DETAILED DESCRIPTION\n\nIn accordance with the present principles, systems and methods are provided to prevent frost damage to crops or other plants in an outdoor environment. The present principles are especially applicable to vineyards, berry farms, citrus farms and other commercial crops. Vineyards will be employed herein as an example. In one embodiment, a system dispenses water uniformly in accordance with conditions measured using environmental variables from a weather station and, in particular, related to air inversion phenomena, which can result in very low temperatures in vineyards in early spring. These conditions may be measured regionally and assume that data will be very similar across the whole region, e.g., over multiple farms, or derived by monitoring/forecasting weather; however, crop freeze may be dependent on very local conditions.\n\nThe systems and methods address frost potential in different ways. For example, the land can be irrigated a day ahead of a frost to 

In [39]:
df.loc[6532]['text']

"DETAILED DESCRIPTION\n\nIn accordance with the present principles, systems and methods are provided to prevent frost damage to crops or other plants in an outdoor environment. The present principles are especially applicable to vineyards, berry farms, citrus farms and other commercial crops. Vineyards will be employed herein as an example. In one embodiment, a system dispenses water uniformly in accordance with conditions measured using environmental variables from a weather station and, in particular, related to air inversion phenomena, which can result in very low temperatures in vineyards in early spring. These conditions may be measured regionally and assume that data will be very similar across the whole region, e.g., over multiple farms, or derived by monitoring/forecasting weather; however, crop freeze may be dependent on very local conditions.\n\nThe systems and methods address frost potential in different ways. For example, the land can be irrigated a day ahead of a frost to 

In [ ]:
with open('tester_everything.txt') as f:
    